In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [5]:
customer_id = '90'
formatted_attribute = 'Cord Length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
field1='Cord Length [Nom]'
fields = [field1]

In [6]:
df_cur, df_custom=parameters(customer_id, formatted_attribute, field1)
df_custom=custom_field(df_custom,field1,formatted_attribute)

425


In [4]:
# matchcust=df_custom[['external_id',curation_col]]
# print(len(matchcust))
# rounding(df_custom,curation_col)

In [7]:
del df_cur['bucket_id']
del df_cur['product_id']
del df_cur['image_url']
del df_cur['value']
del df_cur['custom_fields']

# General Regex

In [8]:
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Le?n?g?t?h?\b)|()'''
df_cur['matches_length'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'''(?i)((?<!up.to.)(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}He?i?g?h?t?(?! Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Wi?d?t?h?\b)|()'''
df_cur['matches_width'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'''(?i)((?<!\d)(?<!\w )(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Diam?e?t?e?r?\b)|(.{0,20}core.?dia.{0,20})|(\d.{0,5}Core)|()'''
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# df_cur['new_matches'] = df_cur['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'''(?i)((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x?\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)?\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|()'''
df_cur['blank_product'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [9]:
print(len(df_cur[df_cur['matches_length'].astype(str)!='[]']))
print(len(df_cur[df_cur['matches_length'].astype(str)=='[]']))
df_cur['matches_length'].explode().value_counts()

0
425


Series([], Name: matches_length, dtype: int64)

In [15]:
regex_pattern_length=r'''(?i)(.{0,40}cord.{0,40})|()'''
df_cur['match'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
print(len(df_cur[df_cur['match'].astype(str)!='[]']))
df_cur['match'].explode().value_counts()

210


 Cable•Connects with your devices via a cord to ensure worry-free signal quality whi    3
Reinforced "strain" connection prevents cord from detaching•Right-angle plug resists    3
lugs.•Heavy-duty strain relief prolongs cord life.•Non-marking jacket protects walls    3
e 3-prong shrouds•Simply plug the power cord's male prong into a standard outlet, an    3
nality of outlets; gives extra reach to cords.•Features three-prong outlets and circ    3
etal connector tips•Cord wrap included                                                  3
e Control: In-Line�•Plug: 3.5 mm stereo•Cord: 6' Dura-Cord� - chew-resistant, PVC-ja    2
tal-Grade Outlets, Safety Covers, 7 ft. Cord•Patented antimicrobial coating 99.9% ef    2
t cord stays flexible to -67F.•Rugged T*Prene                                           2
al connector tips•Cord wrap included                                                    2
c sound reproduction.•Soft touch rubber cord.                                           2
s redundan

# Curate

In [66]:
from natsort import natsorted
regex_pattern=r'''(?i)(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+).?\s?(?:"|'|''|inc?h?|foot|fe?e?t|”).{0,3}(?:rubber\s?)?cord.?L?e?n?g?t?h?)|(cord.?\s?(?<!\.)(?<!\/)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\s?(?:"|'|''|inc?h?|foot|fe?e?t|”))|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# Filter out the SKUs that dont have numbers
pat_number=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(pat_number, str(x)))
lengthNew=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]
del lengthNew['new_matches']

lengthNew['matches']=lengthNew['matches'].apply(lambda x: natsorted(x)).apply(lambda x: sorted(x))
lengthNew['matchez']=lengthNew['matches'].apply(lambda x: re.sub(r'''(?i)(\s?(?:\\?\"|\”|inc?h?|''|�)\W{0,2}Cord)''',' in',str(x))).apply(lambda x: re.sub(r"(?i)\[|\]|'|\\|•|cord[\.\,](?![\s\w\d])|cord\,(?![\s\w\d])|(?<!\d\s)in[\:\s]?|(cord\slength)|length|le?n?g?t?h?|cord|rubber|\:",'',str(x)))
lengthNew['matchez']=lengthNew['matchez'].apply(lambda x: re.sub(r"-",' ',str(x))).apply(lambda x: re.sub(r'(?i)(\sft\.\s?c?o?r?d?)|((?<=\d)ft\s?c?o?r?d?)|(cord\sft)|feet|FE?E?T|foot',' ft',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: re.sub(r'cord\,(?<!\d)(?<!\")','',str(x)))
lengthNew['matchez']=lengthNew['matchez'].apply(lambda x: re.sub(r'.+\"\,|\,\".+|(?<!\")(?<!\w)(?<!\d)\,','',str(x))).apply(lambda x: re.sub(r'\"?\s{0,3}6\s{0,3}\"| 6"','6 ft',str(x))).apply(lambda x: re.sub(r'"15  "|"15   "|"15 "','15 ft',str(x))).apply(lambda x: re.sub(r'  5"','5 ft',str(x)))
lengthNew['matchez']=lengthNew['matchez'].apply(lambda x: re.sub(r'"8\s{0,3}"','8 ft',str(x))).apply(lambda x: re.sub(r'(?i)\"?\s{0,3}7\s{0,3}\"','7 ft',str(x))).apply(lambda x: re.sub(r'\s{2,3}',' ',str(x))).apply(lambda x: re.sub(r'ftft','ft',str(x)))
lengthNew['matchez']=lengthNew['matchez'].apply(lambda x: re.sub(r'(?<![\w\d])\s|\,(?![\w\d])','',str(x)))


curate(lengthNew, 'matchez', formatted_attribute, 'a-eghj-mo-su-z')
print(len(lengthNew))
matchesnew=lengthNew[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()
rounding(lengthNew, 'matchez','a-eghj-mo-su-z')

144


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, matches_length, matches_height, matches_width, matches_diameter, blank_product, blank_long, match, matches, matchez, Q:cord_length, rounding]
Index: []

In [68]:
lengthNew

buckets external_id  \
29   Extension/Power Supply Cords  5637175997   
30   Extension/Power Supply Cords  5637175998   
31   Extension/Power Supply Cords  5637175969   
32   Extension/Power Supply Cords  5637175970   
33   Extension/Power Supply Cords  5637175972   
34   Extension/Power Supply Cords  5637175973   
35   Extension/Power Supply Cords  5637175991   
36   Extension/Power Supply Cords  5637175992   
37   Extension/Power Supply Cords  5637175993   
38   Extension/Power Supply Cords  5637175994   
39   Extension/Power Supply Cords  5637175995   
41   Extension/Power Supply Cords  5637175999   
42   Extension/Power Supply Cords  5637176000   
43   Extension/Power Supply Cords  5637176001   
44   Extension/Power Supply Cords  5637176003   
45   Extension/Power Supply Cords  5637176004   
46   Extension/Power Supply Cords  5637176005   
47   Extension/Power Supply Cords  5637176006   
48   Extension/Power Supply Cords  5637176007   
49   Extension/Power Supply Cords  5637176008   
50   Extension/Power Supply Cords  5637193135   
51   Extension/Power Supply Cords  5637193138   
52   Extension/Power Supply Cords  5637192915   
53   Extension/Power Supply Cords  5637186530   
54   Extension/Power Supply Cords  5637209686   
55   Extension/Power Supply Cords  5637209687   
56   Extension/Power Supply Cords  5637190934   
60   Extension/Power Supply Cords  5637190938   
61   Extension/Power Supply Cords  5637192425   
62   Extension/Power Supply Cords  5637192424   
63   Extension/Power Supply Cords  5637192426   
64   Extension/Power Supply Cords  5637192427   
65   Extension/Power Supply Cords  5637192428   
66   Extension/Power Supply Cords  5637192429   
67   Extension/Power Supply Cords  5637192430   
68   Extension/Power Supply Cords  5637192431   
69   Extension/Power Supply Cords  5637204688   
70   Extension/Power Supply Cords  5637213750   
71   Extension/Power Supply Cords  5637213751   
72   Extension/Power Supply Cords  5637213752   
73   Extension/Power Supply Cords  5637213754   
75   Extension/Power Supply Cords  5637216185   
76   Extension/Power Supply Cords  5637218965   
83   Extension/Power Supply Cords  5637213746   
84   Extension/Power Supply Cords  5637217480   
86   Extension/Power Supply Cords  5637245284   
88   Extension/Power Supply Cords  5637245286   
89   Extension/Power Supply Cords  5637245287   
90   Extension/Power Supply Cords  5637245292   
91   Extension/Power Supply Cords  5637245293   
92   Extension/Power Supply Cords  5637245294   
95   Extension/Power Supply Cords  5637299328   
96   Extension/Power Supply Cords  5637295937   
98   Extension/Power Supply Cords  5637296717   
100  Extension/Power Supply Cords  5637295469   
101  Extension/Power Supply Cords  5637295489   
102  Extension/Power Supply Cords  5637297500   
103  Extension/Power Supply Cords  5637306478   
104  Extension/Power Supply Cords  5637315675   
105  Extension/Power Supply Cords  5637314882   
106  Extension/Power Supply Cords  5637330776   
107  Extension/Power Supply Cords  5637336575   
110  Extension/Power Supply Cords  5637336593   
112  Extension/Power Supply Cords  5637352024   
113  Extension/Power Supply Cords  5637358390   
114  Extension/Power Supply Cords  5637358391   
115  Extension/Power Supply Cords  5637358474   
116  Extension/Power Supply Cords  5637358496   
117  Extension/Power Supply Cords  5637362747   
118  Extension/Power Supply Cords  5637362748   
119  Extension/Power Supply Cords  5637373810   
121  Extension/Power Supply Cords  5637374458   
123  Extension/Power Supply Cords  5637381928   
125  Extension/Power Supply Cords  5637401733   
126  Extension/Power Supply Cords  5637402551   
129  Extension/Power Supply Cords  5637409771   
130  Extension/Power Supply Cords  5637414702   
132  Extension/Power Supply Cords  5637413993   
133  Extension/Power Supply Cords  5637422501   
134  Extension/Power Supply Cords  5637422502   
137  Extension/Power Supply Cords  5637440587   
138  Ext

# N/A Values

In [69]:
x=df_cur[df_cur['matches'].astype(str)=='[]']
print(len(x[x['blank_long'].astype(str)!='[]']))
x[x['blank_long'].astype(str)!='[]'][0:500]

132


buckets external_id  \
0      Mice, Touchpads & Pointers  5637192560   
1      Mice, Touchpads & Pointers  5637218909   
2      Mice, Touchpads & Pointers  5637211298   
3      Mice, Touchpads & Pointers  5637245099   
5      Mice, Touchpads & Pointers  5637299678   
6      Mice, Touchpads & Pointers  5637299679   
7      Mice, Touchpads & Pointers  5637314397   
8      Mice, Touchpads & Pointers  5637314398   
9      Mice, Touchpads & Pointers  5637314399   
10     Mice, Touchpads & Pointers  5637345960   
12     Mice, Touchpads & Pointers  5637441675   
13     Mice, Touchpads & Pointers  5637162488   
15     Mice, Touchpads & Pointers  5637169326   
20     Mice, Touchpads & Pointers  5637490388   
21     Mice, Touchpads & Pointers  5637492052   
22     Mice, Touchpads & Pointers  5637492053   
23     Mice, Touchpads & Pointers  5637509734   
26     Mice, Touchpads & Pointers  5637546861   
27     Mice, Touchpads & Pointers  5637552931   
58   Extension/Power Supply Cords  5637190936   
59   Extension/Power Supply Cords  5637190937   
93   Extension/Power Supply Cords  5637233842   
94   Extension/Power Supply Cords  5637301678   
97   Extension/Power Supply Cords  5637296627   
99   Extension/Power Supply Cords  5637296777   
108  Extension/Power Supply Cords  5637336577   
120  Extension/Power Supply Cords  5637372911   
148  Extension/Power Supply Cords  5637155693   
149  Extension/Power Supply Cords  5637155694   
150  Extension/Power Supply Cords  5637155695   
151  Extension/Power Supply Cords  5637155696   
160  Extension/Power Supply Cords  5637498186   
161  Extension/Power Supply Cords  5637498187   
162  Extension/Power Supply Cords  5637498188   
163  Extension/Power Supply Cords  5637498189   
164  Extension/Power Supply Cords  5637498191   
165  Extension/Power Supply Cords  5637498206   
166  Extension/Power Supply Cords  5637498207   
167  Extension/Power Supply Cords  5637498166   
168  Extension/Power Supply Cords  5637498167   
169  Extension/Power Supply Cords  5637498169   
170  Extension/Power Supply Cords  5637498173   
171  Extension/Power Supply Cords  5637498174   
172  Extension/Power Supply Cords  5637498175   
173  Extension/Power Supply Cords  5637505295   
176  Extension/Power Supply Cords  5637515818   
183  Extension/Power Supply Cords  5637551452   
191  Extension/Power Supply Cords  5637561364   
192  Extension/Power Supply Cords  5637561366   
195  Extension/Power Supply Cords  5637561369   
197  Extension/Power Supply Cords  5637561371   
198  Extension/Power Supply Cords  5637561372   
216  Extension/Power Supply Cords  5637562188   
218  Extension/Power Supply Cords  5637564690   
219  Extension/Power Supply Cords  5637564691   
221  Extension/Power Supply Cords  5637564716   
223  Extension/Power Supply Cords  5637564703   
240     Cables, Cords, & Adapters  5637175957   
241     Cables, Cords, & Adapters  5637259463   
243     Cables, Cords, & Adapters  5637323036   
245     Cables, Cords, & Adapters  5637343066   
248     Cables, Cords, & Adapters  5637381370   
249     Cables, Cords, & Adapters  5637382438   
250     Cables, Cords, & Adapters  5637391470   
251     Cables, Cords, & Adapters  5637404200   
252     Cables, Cords, & Adapters  5637402239   
253     Cables, Cords, & Adapters  5637402240   
254     Cables, Cords, & Adapters  5637402241   
255     Cables, Cords, & Adapters  5637402815   
256     Cables, Cords, & Adapters  5637402816   
257     Cables, Cords, & Adapters  5637438190   
258     Cables, Cords, & Adapters  5637460705   
259     Cables, Cords, & Adapters  5637460707   
263     Cables, Cords, & Adapters  5637490923   
264     Cables, Cords, & Adapters  5637490924   
265     Cables, Cords, & Adapters  5637490925   
266     Cables, Cords, & Adapters  5637494186   
267     Cables, Cords, & Adapters  5637494187   
268     Cables, Cords, & Adapters  5637494188   
269     Cables, Cords, & Adapters  5637494189   
270     Cables, Cords, & Adapters  5637494190   
271     

In [73]:
df_na=df_cur[df_cur['matches'].astype(str)=='[]']
# regex_pattern_na=r'''(?i)((?<!Cord Measures )(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)['"”].{0,2}(?:[LWHD]|diam)?)|(measure)|()'''
# df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

# df_na=df_na[(df_na['na_matches'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
df_na

281


buckets external_id  \
0      Mice, Touchpads & Pointers  5637192560   
1      Mice, Touchpads & Pointers  5637218909   
2      Mice, Touchpads & Pointers  5637211298   
3      Mice, Touchpads & Pointers  5637245099   
4      Mice, Touchpads & Pointers  5637282714   
5      Mice, Touchpads & Pointers  5637299678   
6      Mice, Touchpads & Pointers  5637299679   
7      Mice, Touchpads & Pointers  5637314397   
8      Mice, Touchpads & Pointers  5637314398   
9      Mice, Touchpads & Pointers  5637314399   
10     Mice, Touchpads & Pointers  5637345960   
11     Mice, Touchpads & Pointers  5637345864   
12     Mice, Touchpads & Pointers  5637441675   
13     Mice, Touchpads & Pointers  5637162488   
14     Mice, Touchpads & Pointers  5637169325   
15     Mice, Touchpads & Pointers  5637169326   
16     Mice, Touchpads & Pointers  5637169331   
17     Mice, Touchpads & Pointers  5637169335   
18     Mice, Touchpads & Pointers  5637169338   
19     Mice, Touchpads & Pointers  5637169339   
20     Mice, Touchpads & Pointers  5637490388   
21     Mice, Touchpads & Pointers  5637492052   
22     Mice, Touchpads & Pointers  5637492053   
23     Mice, Touchpads & Pointers  5637509734   
24     Mice, Touchpads & Pointers  5637551433   
25     Mice, Touchpads & Pointers  5637551434   
26     Mice, Touchpads & Pointers  5637546861   
27     Mice, Touchpads & Pointers  5637552931   
28     Mice, Touchpads & Pointers  5637556979   
40   Extension/Power Supply Cords  5637175996   
57   Extension/Power Supply Cords  5637190935   
58   Extension/Power Supply Cords  5637190936   
59   Extension/Power Supply Cords  5637190937   
74   Extension/Power Supply Cords  5637213757   
77   Extension/Power Supply Cords  5637218975   
78   Extension/Power Supply Cords  5637219033   
79   Extension/Power Supply Cords  5637213715   
80   Extension/Power Supply Cords  5637213718   
81   Extension/Power Supply Cords  5637213719   
82   Extension/Power Supply Cords  5637213720   
85   Extension/Power Supply Cords  5637220609   
87   Extension/Power Supply Cords  5637245285   
93   Extension/Power Supply Cords  5637233842   
94   Extension/Power Supply Cords  5637301678   
97   Extension/Power Supply Cords  5637296627   
99   Extension/Power Supply Cords  5637296777   
108  Extension/Power Supply Cords  5637336577   
109  Extension/Power Supply Cords  5637336591   
111  Extension/Power Supply Cords  5637343053   
120  Extension/Power Supply Cords  5637372911   
122  Extension/Power Supply Cords  5637381929   
124  Extension/Power Supply Cords  5637401724   
127  Extension/Power Supply Cords  5637402249   
128  Extension/Power Supply Cords  5637402812   
131  Extension/Power Supply Cords  5637416414   
135  Extension/Power Supply Cords  5637431153   
136  Extension/Power Supply Cords  5637439452   
148  Extension/Power Supply Cords  5637155693   
149  Extension/Power Supply Cords  5637155694   
150  Extension/Power Supply Cords  5637155695   
151  Extension/Power Supply Cords  5637155696   
160  Extension/Power Supply Cords  5637498186   
161  Extension/Power Supply Cords  5637498187   
162  Extension/Power Supply Cords  5637498188   
163  Extension/Power Supply Cords  5637498189   
164  Extension/Power Supply Cords  5637498191   
165  Extension/Power Supply Cords  5637498206   
166  Extension/Power Supply Cords  5637498207   
167  Extension/Power Supply Cords  5637498166   
168  Extension/Power Supply Cords  5637498167   
169  Extension/Power Supply Cords  5637498169   
170  Extension/Power Supply Cords  5637498173   
171  Extension/Power Supply Cords  5637498174   
172  Extension/Power Supply Cords  5637498175   
173  Extension/Power Supply Cords  5637505295   
175  Extension/Power Supply Cords  5637511454   
176  Extension/Power Supply Cords  5637515818   
177  Extension/Power Supply Cords  5637526790   
178  Extension/Power Supply Cords  5637533691   
183  Extension/Power Supply Cords  5637551452   
191  Extension/Power Supply Cords  5637561364   
192  Ext

In [74]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnew))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

matchesnew
matchesna


In [ ]:
stop

# send to the folder for upload

In [75]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesnew)

In [76]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

# looks_good('WBMason', attribute, matchcust)

In [77]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesna)